In [ ]:
# default_exp core

In [ ]:
#hide
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
#hide
%cd "/content/drive/MyDrive/Coding/ModelAssistedLabel"

/content/drive/MyDrive/Coding/ModelAssistedLabel


# Trainer
> Class wrapper for train.py

In [ ]:
#export
import os

class Trainer():
  def __init__(self, name, yaml_file = "./models/custom_yolov5s.yaml"):
    self.yaml_file = yaml_file
    self.name = name

  def write_yaml(self):
    import yaml
    with open("data.yaml", 'r') as stream:
      self.num_classes = str(yaml.safe_load(stream)['nc'])
    if os.path.exists(self.yaml_file):
      os.remove(self.yaml_file)

    template = f"""# parameters
    nc: {self.num_classes}  # number of classes
    depth_multiple: 0.33  # model depth multiple
    width_multiple: 0.50  # layer channel multiple

    # anchors
    anchors:
      - [10,13, 16,30, 33,23]  # P3/8
      - [30,61, 62,45, 59,119]  # P4/16
      - [116,90, 156,198, 373,326]  # P5/32

    # YOLOv5 backbone
    backbone:
      # [from, number, module, args]
      [[-1, 1, Focus, [64, 3]],  # 0-P1/2
      [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
      [-1, 3, BottleneckCSP, [128]],
      [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
      [-1, 9, BottleneckCSP, [256]],
      [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
      [-1, 9, BottleneckCSP, [512]],
      [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
      [-1, 1, SPP, [1024, [5, 9, 13]]],
      [-1, 3, BottleneckCSP, [1024, False]],  # 9
      ]

    # YOLOv5 head
    head:
      [[-1, 1, Conv, [512, 1, 1]],
      [-1, 1, nn.Upsample, [None, 2, 'nearest']],
      [[-1, 6], 1, Concat, [1]],  # cat backbone P4
      [-1, 3, BottleneckCSP, [512, False]],  # 13

      [-1, 1, Conv, [256, 1, 1]],
      [-1, 1, nn.Upsample, [None, 2, 'nearest']],
      [[-1, 4], 1, Concat, [1]],  # cat backbone P3
      [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

      [-1, 1, Conv, [256, 3, 2]],
      [[-1, 14], 1, Concat, [1]],  # cat head P4
      [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

      [-1, 1, Conv, [512, 3, 2]],
      [[-1, 10], 1, Concat, [1]],  # cat head P5
      [-1, 3, BottleneckCSP, [1024, 
      False]],  # 23 (P5/32-large)

      [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
      ]"""

    f = open(self.yaml_file,"w+")
    f.writelines(template)
    f.close()

  def train(self, epochs):
    self.write_yaml()
    os.chdir("yolov5")
    os.system("pip install -r requirements.txt")
    os.system(f"python train.py --img 416 --batch 16 --epochs {epochs} --data '../data.yaml' --cfg {self.yaml_file} --weights '' --name {self.name}  --cache")
    os.chdir("..")